In [1]:
import sys
import os
sys.path.append('../../')

%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import numpy as np

import matplotlib.image as mpimg
import matplotlib.pyplot as plt

from skimage.io import imread

from gen.load_data import load_data

In [3]:
from sklearn.utils import shuffle

train_df, valid_df, test_df = load_data('../../data')


print(train_df.head())


                                               image                   id  \
0  ../../data/Train/CameraRGB/episode_0002_000287...  episode_0002_000287   
1  ../../data/Train/CameraRGB/episode_0008_000112...  episode_0008_000112   
2                 ../../data/Train/CameraRGB/804.png                  804   
3  ../../data/Train/CameraRGB/episode_0008_000286...  episode_0008_000286   
4  ../../data/Train/CameraRGB/episode_0014_000061...  episode_0014_000061   

                                               label  
0  ../../data/Train/CameraSeg/episode_0002_000287...  
1  ../../data/Train/CameraSeg/episode_0008_000112...  
2                 ../../data/Train/CameraSeg/804.png  
3  ../../data/Train/CameraSeg/episode_0008_000286...  
4  ../../data/Train/CameraSeg/episode_0014_000061...  


In [4]:
from models.segnet import model_segnetVGG16


model = model_segnetVGG16(3, image_shape=(416, 544, 3))
model.summary()

/home/faisal/anaconda3/envs/ai/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 416, 544, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 416, 544, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 416, 544, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 208, 272, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 208, 272, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 208, 272, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 104, 136, 128)     0         
__________

In [8]:
from gen.datagen import oversample_generator_from_df, balanced_generator_from_df

GPUS = 2
BATCH_SIZE = 8 * GPUS
model_dir = '../../saved_models/segnet/segnet_v5/'

if not os.path.exists(model_dir):
    os.mkdir(model_dir)

from gen.generators import gen_func, preprocess_label, preprocess_multi_label
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.vgg16 import VGG16, preprocess_input

rgb_gen = ImageDataGenerator()
lab_gen = ImageDataGenerator()

train_gen = gen_func(train_df, rgb_gen, lab_gen, image_size=(600, 800), target_size=(416, 544), batch_size = BATCH_SIZE, perturb=True)
valid_gen = gen_func(valid_df, rgb_gen, lab_gen, image_size=(600, 800), target_size=(416, 544), batch_size = BATCH_SIZE)

In [9]:
from keras.utils.training_utils import multi_gpu_model

model_gpu = multi_gpu_model(model, GPUS)

model_gpu.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [10]:
from train import train_nn

m = train_df.shape[0]
history = train_nn(model_gpu, 
                   train_gen, 
                   valid_gen, 
                   training_size=train_df.shape[0], 
                   batch_size=BATCH_SIZE,
                   validation_size=valid_df.shape[0],
                   output_path=model_dir, 
                   epochs=100,
                   gpus = GPUS)

## Ignore next message from keras, values are replaced anyways
## Ignore next message from keras, values are replaced anyways
Epoch 1/100
Found 0 images belonging to 0 classes.
Found 0 images belonging to 0 classes.
Reinserting dataframe: 1683 images
## Ignore next message from keras, values are replaced anyways
Found 0 images belonging to 0 classes.
Reinserting dataframe: 6159 images
## Ignore next message from keras, values are replaced anyways
Found 0 images belonging to 0 classes.
Reinserting dataframe: 1683 images
Reinserting dataframe: 6159 images
192/192 [==============================] - 186s 967ms/step - loss: 0.1213 - acc: 0.9615 - val_loss: 0.0933 - val_acc: 0.9698

Epoch 00001: val_loss improved from inf to 0.09331, saving model to ../../saved_models/segnet/segnet_v5//model.hdf5
Epoch 2/100
192/192 [==============================] - 178s 925ms/step - loss: 0.0571 - acc: 0.9787 - val_loss: 0.1015 - val_acc: 0.9671

Epoch 00002: val_loss did not improve
Epoch 3/100
192/192 [=

KeyboardInterrupt: 

In [ ]:
# # summarize history for accuracy
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [11]:
model_gpu.load_weights('../../saved_models/segnet/segnet_v5//model.hdf5')
# model.reset_states()
model.save('../../saved_models/segnet/segnet_v5//model_saved.h5')